In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


In [ ]:
df = pd.read_csv('/content/training1600000.csv',encoding = "ISO-8859-1",header=None)

In [ ]:

df.columns = ["polarity",'id','date','query','user','tweet']
df

,polarity,id,date,query,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


# Missing Values

In [ ]:
df.isnull().sum()

polarity    0
id          0
date        0
query       0
user        0
tweet       0
dtype: int64

## Text Cleaning

In [ ]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+',' ',text)
    text = re.sub('[^a-zA-Z]',' ',text)
    text = word_tokenize(text)
    text = [item for item in text if item not in stop_words]
    text = [lemma.lemmatize(w) for w in text]
    text = [i for i in text if len(i)>2]
    text = ' '.join(text)
    return text

In [ ]:
stop_words = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

In [ ]:
# df_new = df[0:100000]
df_new = df.sample(n=100000)


In [ ]:
df_new['polarity'].value_counts()

4    50017
0    49983
Name: polarity, dtype: int64

In [ ]:
df_new['clean_tweet'] = df_new['tweet'].apply(clean_text)

In [ ]:
x_train = df_new['clean_tweet']

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df=10)

x_train_tfidf = pd.DataFrame(tfidf_vectorizer.fit_transform(x_train).toarray(), columns=tfidf_vectorizer.get_feature_names_out())

display(x_train_tfidf.shape)
display(x_train_tfidf.head())

(100000, 5704)

,aaah,aah,aaron,abandoned,abby,abc,ability,abit,able,absolute,...,zackalltimelow,zealand,zero,zoe,zombie,zomg,zone,zoo,zune,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
model = LinearSVC()

In [ ]:
y_train = df_new.polarity

In [ ]:
# y_train

In [ ]:
df_test = pd.read_csv('/content/testdata.manual.2009.06.14.csv',header=None)

In [ ]:
df_test.columns = ["polarity",'id','date','query','user','tweet']
df_test

,polarity,id,date,query,user,tweet
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
...,...,...,...,...,...,...
493,2,14072,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...
494,0,14073,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat..."
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


In [ ]:
df_test['clean_tweet'] = df_test['tweet'].apply(clean_text)

In [ ]:
x_test = df_test['clean_tweet']
x_test_tfidf = pd.DataFrame(tfidf_vectorizer.transform(x_test).toarray(), columns=tfidf_vectorizer.get_feature_names_out())


In [ ]:
y_test = df_test['polarity']

In [ ]:
model.fit(x_train_tfidf, y_train)
results = {
    'accuracy': [],
    'f1': []
}
y_pred = model.predict(x_test_tfidf)
results['accuracy'].append(accuracy_score(y_test, y_pred))
results['f1'].append(f1_score(y_test, y_pred, average=None))

In [ ]:
results

{'accuracy': [0.570281124497992],
 'f1': [array([0.71823204, 0.        , 0.62222222])]}

In [ ]:
df_filtered_test = df_test[df_test['polarity'] != 2]


In [ ]:
x_test_filtered = df_filtered_test['clean_tweet']
x_test_filtered_tfidf = pd.DataFrame(tfidf_vectorizer.transform(x_test_filtered).toarray(), columns=tfidf_vectorizer.get_feature_names_out())


In [ ]:
y_filtered_test = df_filtered_test['polarity']

In [ ]:
len(y_filtered_test)

359

In [ ]:
model.fit(x_train_tfidf, y_train)
results_filtered_SVC = {
    'accuracy': [],
    'f1': []
}
y_pred = model.predict(x_test_filtered_tfidf)
results_filtered_SVC['accuracy'].append(accuracy_score(y_filtered_test, y_pred))
results_filtered_SVC['f1'].append(f1_score(y_filtered_test, y_pred, average=None))

In [ ]:
results_filtered_SVC

{'accuracy': [0.7910863509749304], 'f1': [array([0.7761194 , 0.80417755])]}

In [ ]:
results

{'accuracy': [0.570281124497992],
 'f1': [array([0.71823204, 0.        , 0.62222222])]}

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
model2 = SGDClassifier(max_iter=10000, tol=1e-3)

In [ ]:
model2.fit(x_train_tfidf, y_train)
results_SGD = {
    'accuracy': [],
    'f1': []
}
y_pred = model.predict(x_test_tfidf)
results_SGD['accuracy'].append(accuracy_score(y_test, y_pred))
results_SGD['f1'].append(f1_score(y_test, y_pred, average=None))

In [ ]:
results_SGD

{'accuracy': [0.570281124497992],
 'f1': [array([0.71823204, 0.        , 0.62222222])]}

In [ ]:

model2.fit(x_train_tfidf, y_train)
results_filtered_SGD = {
    'accuracy': [],
    'f1': []
}

y_pred = model.predict(x_test_filtered_tfidf)
results_filtered_SGD['accuracy'].append(accuracy_score(y_filtered_test, y_pred))
results_filtered_SGD['f1'].append(f1_score(y_filtered_test, y_pred, average=None))

In [ ]:
results_filtered_SGD

{'accuracy': [0.7910863509749304], 'f1': [array([0.7761194 , 0.80417755])]}